In [1]:
import math
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from UDFManager import *
from pyudf.rotation import Quaternion

plt.style.use('presentation')

In [2]:
def quaternionToRotationArray(q):
    return [
        [1 - 2 * (q.q2 * q.q2 + q.q3 * q.q3), 2 * (q.q1 * q.q2 + q.q0 * q.q3), 2 * (q.q3 * q.q1 - q.q0 * q.q2)],
        [2 * (q.q1 * q.q2 - q.q0 * q.q3), 1 - 2 * (q.q1 * q.q1 + q.q3 * q.q3), 2 * (q.q2 * q.q3 + q.q0 * q.q1)],
        [2 * (q.q3 * q.q1 + q.q0 * q.q2), 2 * (q.q2 * q.q3 - q.q0 * q.q1), 1 - 2 * (q.q1 * q.q1 - q.q2 * q.q2)]
    ]

In [3]:
RHO = 1.0
MU  = 1.0
a   = 5.0
h   = a / 2.0
gra = 0.06

gravity = np.array([0.0, 1.0, 0.0]) * (gra * RHO * 4 / 3 * np.pi * (a**3))

In [4]:
gammadot_map = {}
radian_map   = {}
tr_hy_map    = {}
tr_s_map     = {}
tr_bh_map    = {}
tr_cs_map    = {}
tr_cbh_mao   = {}

In [5]:
gammadots = []
radians   = []
tr_hys    = []
tr_ss     = []
tr_bhs    = []
tr_css    = []
tr_cbhs   = []

for i in tqdm(range(5, 155, 5)):
    uobj = UDFManager(f'00/udf/{i:0=3}/output.udf')
    gammadot = uobj.get("constitutive_eq.Shear_Navier_Stokes.External_field.DC.Shear_rate")
    gammadots.append(gammadot)
    nt = uobj.totalRecord()
    
    tr_hy  = np.zeros(nt)  # from udf
    tr_s   = np.zeros(nt)  # from udf
    tr_bh  = np.zeros(nt)  # from udf
    radian = np.zeros(nt)
    
    tr_cs  = np.ones(nt) * (8 * np.pi * MU * (a**3) * 0.5 * gammadot) # calc
    tr_cbh = np.zeros(nt)
    for n in range(nt):
        uobj.jump(n)
        tr_hy[n] = uobj.get("Particles[].torque_hydro")[0][-1]
        tr_s[n]  = uobj.get("Particles[].torque_slip")[0][-1]
        tr_bh[n] = uobj.get("Particles[].torque_bh")[0][-1]
        
        q = Quaternion(*(uobj.get("Particles[].q"))[0])
        rotation_matrix = quaternionToRotationArray(q)[1]
        radian[n] = np.arccos(rotation_matrix[1])
        tr_cbh[n] = (np.cross(rotation_matrix, gravity) * h)[-1]

    tr_hys.append(tr_hy)
    tr_ss.append(tr_s)
    tr_bhs.append(tr_bh)
    
    radians.append(radian)
    tr_css.append(tr_cs)
    tr_cbhs.append(tr_cbh)
#     print(f'shear: {gammadot: 2.4f}\t tr_hy: {tr_hy[-1]: 2.5f}\t tr_s: {tr_s[-1]: 2.5f}\t sum: {tr_hy[-1]+tr_s[-1]: 2.6f} \t tr_bh: {tr_bh[-1]: 2.5f}\t cal_shear: {tr_cs[-1]: 2.5f}\t cal_bh: {tr_cbh[-1]: 2.5f}')

gammadots = np.array(gammadots)
n_gammadots = MU * (a**3) * 0.5 * gammadots * 8 * np.pi * 10

gammadot_map['00'] = gammadots
radian_map['00']   = radians
tr_hy_map['00']    = tr_hys
tr_s_map['00']     = tr_ss
tr_bh_map['00']    = tr_bhs
tr_cs_map['00']    = tr_css
tr_cbh_mao['00']   = tr_cbhs

In [6]:
gammadots = []
radians   = []
tr_hys    = []
tr_ss     = []
tr_bhs    = []
tr_css    = []
tr_cbhs   = []

for i in tqdm(range(5, 155, 5)):
    uobj = UDFManager(f'01/udf/{i:0=3}/output.udf')
    gammadot = uobj.get("constitutive_eq.Shear_Navier_Stokes.External_field.DC.Shear_rate")
    gammadots.append(gammadot)
    nt = uobj.totalRecord()
    
    tr_hy  = np.zeros(nt)  # from udf
    tr_s   = np.zeros(nt)  # from udf
    tr_bh  = np.zeros(nt)  # from udf
    radian = np.zeros(nt)
    
    tr_cs  = np.ones(nt) * (8 * np.pi * MU * (a**3) * 0.5 * gammadot) # calc
    tr_cbh = np.zeros(nt)
    for n in range(nt):
        uobj.jump(n)
        tr_hy[n] = uobj.get("Particles[].torque_hydro")[0][-1]
        tr_s[n]  = uobj.get("Particles[].torque_slip")[0][-1]
        tr_bh[n] = uobj.get("Particles[].torque_bh")[0][-1]
        
        q = Quaternion(*(uobj.get("Particles[].q"))[0])
        rotation_matrix = quaternionToRotationArray(q)[1]
        radian[n] = np.arccos(rotation_matrix[1])
        tr_cbh[n] = (np.cross(rotation_matrix, gravity) * h)[-1]

    tr_hys.append(tr_hy)
    tr_ss.append(tr_s)
    tr_bhs.append(tr_bh)
    
    radians.append(radian)
    tr_css.append(tr_cs)
    tr_cbhs.append(tr_cbh)
#     print(f'shear: {gammadot: 2.4f}\t tr_hy: {tr_hy[-1]: 2.5f}\t tr_s: {tr_s[-1]: 2.5f}\t sum: {tr_hy[-1]+tr_s[-1]: 2.6f} \t tr_bh: {tr_bh[-1]: 2.5f}\t cal_shear: {tr_cs[-1]: 2.5f}\t cal_bh: {tr_cbh[-1]: 2.5f}')

gammadots = np.array(gammadots)
n_gammadots = MU * (a**3) * 0.5 * gammadots * 8 * np.pi * 10

gammadot_map['01'] = gammadots
radian_map['01']   = radians
tr_hy_map['01']    = tr_hys
tr_s_map['01']     = tr_ss
tr_bh_map['01']    = tr_bhs
tr_cs_map['01']    = tr_css
tr_cbh_mao['01']   = tr_cbhs

In [7]:
gammadots = []
radians   = []
tr_hys    = []
tr_ss     = []
tr_bhs    = []
tr_css    = []
tr_cbhs   = []

for i in tqdm(range(5, 155, 5)):
    uobj = UDFManager(f'02/udf/{i:0=3}/output.udf')
    gammadot = uobj.get("constitutive_eq.Shear_Navier_Stokes.External_field.DC.Shear_rate")
    gammadots.append(gammadot)
    nt = uobj.totalRecord()
    
    tr_hy  = np.zeros(nt)  # from udf
    tr_s   = np.zeros(nt)  # from udf
    tr_bh  = np.zeros(nt)  # from udf
    radian = np.zeros(nt)
    
    tr_cs  = np.ones(nt) * (8 * np.pi * MU * (a**3) * 0.5 * gammadot) # calc
    tr_cbh = np.zeros(nt)
    for n in range(nt):
        uobj.jump(n)
        tr_hy[n] = uobj.get("Particles[].torque_hydro")[0][-1]
        tr_s[n]  = uobj.get("Particles[].torque_slip")[0][-1]
        tr_bh[n] = uobj.get("Particles[].torque_bh")[0][-1]
        
        q = Quaternion(*(uobj.get("Particles[].q"))[0])
        rotation_matrix = quaternionToRotationArray(q)[1]
        radian[n] = np.arccos(rotation_matrix[1])
        tr_cbh[n] = (np.cross(rotation_matrix, gravity) * h)[-1]

    tr_hys.append(tr_hy)
    tr_ss.append(tr_s)
    tr_bhs.append(tr_bh)
    
    radians.append(radian)
    tr_css.append(tr_cs)
    tr_cbhs.append(tr_cbh)
#     print(f'shear: {gammadot: 2.4f}\t tr_hy: {tr_hy[-1]: 2.5f}\t tr_s: {tr_s[-1]: 2.5f}\t sum: {tr_hy[-1]+tr_s[-1]: 2.6f} \t tr_bh: {tr_bh[-1]: 2.5f}\t cal_shear: {tr_cs[-1]: 2.5f}\t cal_bh: {tr_cbh[-1]: 2.5f}')

gammadots = np.array(gammadots)
n_gammadots = MU * (a**3) * 0.5 * gammadots * 8 * np.pi * 10

gammadot_map['02'] = gammadots
radian_map['02']   = radians
tr_hy_map['02']    = tr_hys
tr_s_map['02']     = tr_ss
tr_bh_map['02']    = tr_bhs
tr_cs_map['02']    = tr_css
tr_cbh_mao['02']   = tr_cbhs

In [8]:
gammadots = []
radians   = []
tr_hys    = []
tr_ss     = []
tr_bhs    = []
tr_css    = []
tr_cbhs   = []

for i in tqdm(range(5, 155, 5)):
    uobj = UDFManager(f'03/udf/{i:0=3}/output.udf')
    gammadot = uobj.get("constitutive_eq.Shear_Navier_Stokes.External_field.DC.Shear_rate")
    gammadots.append(gammadot)
    nt = uobj.totalRecord()
    
    tr_hy  = np.zeros(nt)  # from udf
    tr_s   = np.zeros(nt)  # from udf
    tr_bh  = np.zeros(nt)  # from udf
    radian = np.zeros(nt)
    
    tr_cs  = np.ones(nt) * (8 * np.pi * MU * (a**3) * 0.5 * gammadot) # calc
    tr_cbh = np.zeros(nt)
    for n in range(nt):
        uobj.jump(n)
        tr_hy[n] = uobj.get("Particles[].torque_hydro")[0][-1]
        tr_s[n]  = uobj.get("Particles[].torque_slip")[0][-1]
        tr_bh[n] = uobj.get("Particles[].torque_bh")[0][-1]
        
        q = Quaternion(*(uobj.get("Particles[].q"))[0])
        rotation_matrix = quaternionToRotationArray(q)[1]
        radian[n] = np.arccos(rotation_matrix[1])
        tr_cbh[n] = (np.cross(rotation_matrix, gravity) * h)[-1]

    tr_hys.append(tr_hy)
    tr_ss.append(tr_s)
    tr_bhs.append(tr_bh)
    
    radians.append(radian)
    tr_css.append(tr_cs)
    tr_cbhs.append(tr_cbh)
#     print(f'shear: {gammadot: 2.4f}\t tr_hy: {tr_hy[-1]: 2.5f}\t tr_s: {tr_s[-1]: 2.5f}\t sum: {tr_hy[-1]+tr_s[-1]: 2.6f} \t tr_bh: {tr_bh[-1]: 2.5f}\t cal_shear: {tr_cs[-1]: 2.5f}\t cal_bh: {tr_cbh[-1]: 2.5f}')

gammadots = np.array(gammadots)
n_gammadots = MU * (a**3) * 0.5 * gammadots * 8 * np.pi * 10

gammadot_map['03'] = gammadots
radian_map['03']   = radians
tr_hy_map['03']    = tr_hys
tr_s_map['03']     = tr_ss
tr_bh_map['03']    = tr_bhs
tr_cs_map['03']    = tr_css
tr_cbh_mao['03']   = tr_cbhs